In [16]:
import dotenv


dotenv.load_dotenv(dotenv_path='secrets.env')

from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.embeddings.embeddings import Embeddings
from langchain_core.vectorstores.base import VectorStore



In [19]:
"""
Define Embeddings and VectorStore
"""

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

client = QdrantClient("http://localhost:6333")

vector_size = len(embeddings.embed_query("sample text"))

if not client.collection_exists("test"):
    client.create_collection(
        collection_name="test",
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name="test",
    embedding=embeddings,
)

In [25]:
"""
Define a record manager
"""
from langchain_classic.indexes import SQLRecordManager, index

record_manager = SQLRecordManager(
    f"elasticsearch/test_index", db_url="sqlite:///record_manager_cache.sqlite",

)
record_manager.create_schema()

# 3. Define your documents
# docs = [
#    Document(page_content="LangChain is great.", metadata={"source": "doc1"}),
#    Document(page_content="Vector stores are useful.", metadata={"source": "doc2"}),
# ]

# result = index(
#    docs,
#    record_manager,
#    vector_store,
#    cleanup="incremental",
#    source_id_key="source",
#    # key_encoder="sha256"
#)



In [27]:
"""
Load documents
"""

from langchain_community.document_loaders.directory import DirectoryLoader

loader = DirectoryLoader("data/ephemeral/spring-doc")
# loader_cls=UnstructuredMarkdownLoader

docs = loader.load()
len(docs)

9

In [29]:
doc = docs[0]
doc

Document(metadata={'source': 'data/ephemeral/spring-doc/spring-amqp.md'}, page_content='Spring Boot Documentation\n\nSource: https://docs.spring.io/spring-amqp/reference/\n\nSpring AMQP\n\nSpring AMQP\n\nThe Spring AMQP project applies core Spring concepts to the development of AMQP-based messaging solutions. We provide a “template” as a high-level abstraction for sending and receiving messages. We also provide support for message-driven POJOs. These libraries facilitate management of AMQP resources while promoting the use of dependency injection and declarative configuration. In all of these cases, you can see similarities to the JMS support in the Spring Framework. For other project-related information, visit the Spring AMQP project homepage.\n\nSpring AMQP\n\nWhat’s New\n\nWhat’s New\n\nChanges in 4.0 Since 3.2\n\nSpring Framework 7.0\n\nThis version requires Spring Framework 7.0.\n\nNull-safety\n\nAs many other Spring portfolio projects, Spring AMQP has been migrated to JSpecify an

In [28]:
"""
Split documents
"""
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(len(all_splits))

14025


In [10]:
result = index(
    all_splits,
    record_manager,
    vector_store,
    cleanup="incremental",
    source_id_key="source",
    # key_encoder="sha256"
)

NameError: name 'embeddings' is not defined